# 08 Custom Presets Guide

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/notebooks/08_workflow_system/08_custom_presets.ipynb)

Custom Preset Guide: Building Domain-Specific Configurations.

This guide demonstrates the kwargs factory pattern for creating domain-specific
workflow configurations in NLSQ. The library provides generic presets (standard,
quality, fast, streaming) that can be customized for any scientific or engineering
domain.

Key patterns covered:
1. Using WORKFLOW_PRESETS dictionary
2. Creating kwargs factory functions
3. Overriding preset settings with fit()
4. Building reusable domain-specific presets
5. Common parameter adjustments by use case

Run this example:
    python examples/scripts/08_workflow_system/08_custom_presets.py

In [1]:
# @title Install NLSQ (run once in Colab)
import sys

if 'google.colab' in sys.modules:
    print("Running in Google Colab - installing NLSQ...")
    !pip install -q nlsq
    print("NLSQ installed successfully!")
else:
    print("Not running in Colab - assuming NLSQ is already installed")

Not running in Colab - assuming NLSQ is already installed


In [2]:
import jax.numpy as jnp
import numpy as np

from nlsq import fit
from nlsq.core.minpack import WORKFLOW_PRESETS

In [3]:
def main():
    print("=" * 70)
    print("Custom Preset Guide: The kwargs Factory Pattern")
    print("=" * 70)
    print()

    # =========================================================================
    # 1. Available Base Presets
    # =========================================================================
    print("1. Available WORKFLOW_PRESETS")
    print("-" * 60)
    print()
    print("NLSQ provides these presets as starting points:")
    print()

    for name in WORKFLOW_PRESETS:
        desc = WORKFLOW_PRESETS[name].get("description", "No description")
        print(f"  - {name}: {desc}")

    # =========================================================================
    # 2. Inspecting Preset Contents
    # =========================================================================
    print()
    print()
    print("2. Inspecting Preset Contents")
    print("-" * 60)
    print()
    print("Each preset is a dictionary of fit() parameters:")
    print()

    for preset_name in ["standard", "quality", "fast"]:
        if preset_name in WORKFLOW_PRESETS:
            print(f"  '{preset_name}' preset:")
            for key, value in list(WORKFLOW_PRESETS[preset_name].items())[:5]:
                print(f"    {key}: {value}")
            print()

    # =========================================================================
    # 3. Creating Custom Presets with kwargs Factory
    # =========================================================================
    print()
    print("3. Creating Custom Presets with kwargs Factory")
    print("-" * 60)
    print()
    print("The recommended pattern is to create factory functions that return kwargs:")
    print()
    print("  def create_my_preset() -> dict:")
    print("      return {")
    print("          'workflow': 'standard',")
    print("          'multistart': True,")
    print("          'n_starts': 20,")
    print("      'gtol': 1e-10,")
    print("      }")
    print()
    print("  # Usage:")
    print("  popt, pcov = fit(model, x, y, **create_my_preset())")

    # =========================================================================
    # 4. Common Override Patterns
    # =========================================================================
    print()
    print()
    print("4. Common Override Patterns")
    print("-" * 60)

    # Pattern A: Increase multi-start coverage
    print()
    print("  Pattern A: Increase multi-start coverage")
    print("  (for models with multiple local minima)")
    print()

    def create_multistart_preset() -> dict:
        return {
            "workflow": "standard",
            "multistart": True,
            "n_starts": 30,
            "sampler": "sobol",
        }

    config_a = create_multistart_preset()
    print(f"    n_starts: {config_a['n_starts']}")
    print(f"    sampler:  {config_a['sampler']}")

    # Pattern B: Tighten tolerances
    print()
    print("  Pattern B: Tighten tolerances")
    print("  (for high-precision structural parameters)")
    print()

    def create_precision_preset() -> dict:
        return {
            "workflow": "quality",
            "gtol": 1e-12,
            "ftol": 1e-12,
            "xtol": 1e-12,
        }

    config_b = create_precision_preset()
    print(f"    gtol: {config_b['gtol']}")
    print(f"    ftol: {config_b['ftol']}")
    print(f"    xtol: {config_b['xtol']}")

    # Pattern C: Memory-constrained environment
    print()
    print("  Pattern C: Memory-constrained fitting")
    print("  (for systems with limited RAM)")
    print()

    def create_memory_constrained_preset() -> dict:
        return {
            "workflow": "streaming",
            "multistart": False,
        }

    config_c = create_memory_constrained_preset()
    print(f"    workflow: {config_c['workflow']}")

    # =========================================================================
    # 5. Creating Reusable Preset Factories
    # =========================================================================
    print()
    print()
    print("5. Creating Reusable Preset Factories")
    print("-" * 60)
    print()
    print("  Define functions that return customized kwargs for your domain:")
    print()

    def create_spectroscopy_preset(high_resolution: bool = False) -> dict:
        """Create preset for spectroscopic peak fitting."""
        return {
            "workflow": "quality" if high_resolution else "standard",
            "multistart": True,
            "n_starts": 15,
            "sampler": "lhs",
            "gtol": 1e-10 if high_resolution else 1e-8,
        }

    def create_timeseries_preset(n_points: int) -> dict:
        """Create preset for time series analysis."""
        if n_points > 1_000_000:
            workflow = "streaming"
        else:
            workflow = "standard"

        return {
            "workflow": workflow,
            "multistart": True,
            "n_starts": 10,
        }

    def create_optimization_preset(n_params: int) -> dict:
        """Create preset based on parameter count."""
        n_starts = max(10, n_params * 3)
        return {
            "workflow": "standard",
            "multistart": True,
            "n_starts": n_starts,
            "sampler": "sobol",
        }

    # Demonstrate factories
    print("  Examples:")
    spec_config = create_spectroscopy_preset(high_resolution=True)
    print(f"    Spectroscopy (high-res): gtol={spec_config['gtol']}, n_starts={spec_config['n_starts']}")

    ts_config = create_timeseries_preset(n_points=500_000)
    print(f"    Time series (500K pts):  workflow={ts_config['workflow']}")

    opt_config = create_optimization_preset(n_params=8)
    print(f"    8-param optimization:    n_starts={opt_config['n_starts']}")

    # =========================================================================
    # 6. Complete Example: Domain-Specific Fitting
    # =========================================================================
    print()
    print()
    print("6. Complete Example: Domain-Specific Fitting")
    print("-" * 60)
    print()

    def create_my_domain_preset() -> dict:
        """Create a preset for my specific application."""
        return {
            "workflow": "standard",
            "gtol": 1e-9,
            "ftol": 1e-9,
            "xtol": 1e-9,
            "multistart": True,
            "n_starts": 15,
            "sampler": "sobol",
        }

    kwargs = create_my_domain_preset()

    print("  Custom domain preset configuration:")
    for key, value in kwargs.items():
        print(f"    {key}: {value}")
    print()

    # Simple test fit
    print("  Testing with exponential decay fit...")

    def exponential(x, a, b, c):
        return a * jnp.exp(-b * x) + c

    np.random.seed(42)
    x_data = np.linspace(0, 5, 100)
    y_true = 2.5 * np.exp(-1.3 * x_data) + 0.5
    y_data = y_true + 0.1 * np.random.randn(100)

    popt, pcov = fit(
        exponential,
        x_data,
        y_data,
        p0=[1.0, 1.0, 0.0],
        bounds=([0.1, 0.1, -1.0], [10.0, 10.0, 2.0]),
        **kwargs,
    )

    print(f"    Fitted parameters: a={popt[0]:.4f}, b={popt[1]:.4f}, c={popt[2]:.4f}")
    print("    True parameters:   a=2.5000, b=1.3000, c=0.5000")

    # =========================================================================
    # 7. Summary: Key Takeaways
    # =========================================================================
    print()
    print()
    print("=" * 70)
    print("Summary: Key Takeaways")
    print("=" * 70)
    print()
    print("1. Use WORKFLOW_PRESETS to see available options:")
    print("   from nlsq.core.minpack import WORKFLOW_PRESETS")
    print()
    print("2. Create kwargs factory functions for your domain:")
    print("   def create_my_preset() -> dict: ...")
    print()
    print("3. Use with fit() via kwargs unpacking:")
    print("   popt, pcov = fit(model, x, y, **create_my_preset())")
    print()
    print("4. Common customizations:")
    print("   - workflow: 'fast', 'standard', 'quality', 'streaming'")
    print("   - n_starts: More for complex models, fewer for simple ones")
    print("   - sampler: 'sobol' for better coverage, 'lhs' for efficiency")
    print("   - tolerances: Tighter for structural params, looser for rates")
    print()
    print("5. Override individual parameters:")
    print("   fit(model, x, y, workflow='standard', gtol=1e-12)")
    print()

In [4]:
if __name__ == "__main__":
    main()

INFO:nlsq.multi_start:Generating 15 starting points using sobol n_starts=15 | sampler=sobol | center_on_p0=True


Custom Preset Guide: The kwargs Factory Pattern

1. Available WORKFLOW_PRESETS
------------------------------------------------------------

NLSQ provides these presets as starting points:

  - standard: Standard curve_fit() with default tolerances
  - quality: Highest precision with multi-start and tighter tolerances
  - fast: Speed-optimized with looser tolerances
  - large_robust: Chunked processing with multi-start for large datasets
  - streaming: AdaptiveHybridStreamingOptimizer for huge datasets
  - hpc_distributed: Multi-GPU/node configuration for HPC clusters


2. Inspecting Preset Contents
------------------------------------------------------------

Each preset is a dictionary of fit() parameters:

  'standard' preset:
    description: Standard curve_fit() with default tolerances
    tier: STANDARD
    enable_multistart: False
    gtol: 1e-08
    ftol: 1e-08

  'quality' preset:
    description: Highest precision with multi-start and tighter tolerances
    tier: STANDARD
   

INFO:nlsq.multi_start:Evaluating 15 starting points


INFO:nlsq.curve_fit:Starting curve fit n_params=3 | n_data_points=100 | method=trf | solver=auto | batch_size=None | has_bounds=True | dynamic_sizing=False


INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=3 | loss=linear | ftol=1.0000e-09 | xtol=1.0000e-09 | gtol=1.0000e-09


...

INFO:nlsq.least_squares:Convergence reason=`ftol` termination condition is satisfied. | iterations=6 | final_cost=0.3851 | elapsed=0.084s | final_gradient_norm=3.8199e-09


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit elapsed=0.166722s


INFO:nlsq.curve_fit:Curve fit completed total_time=0.1667 | final_cost=0.7702 | covariance_warning=False


INFO:nlsq.multi_start:Best starting point: loss=0.770178 best_loss=0.7702 | best_params=[2.576556916160811, 1.4017396918476726, 0.5058036829069804]


    Fitted parameters: a=2.5766, b=1.4017, c=0.5058
    True parameters:   a=2.5000, b=1.3000, c=0.5000


Summary: Key Takeaways

1. Use WORKFLOW_PRESETS to see available options:
   from nlsq.core.minpack import WORKFLOW_PRESETS

2. Create kwargs factory functions for your domain:
   def create_my_preset() -> dict: ...

3. Use with fit() via kwargs unpacking:
   popt, pcov = fit(model, x, y, **create_my_preset())

4. Common customizations:
   - workflow: 'fast', 'standard', 'quality', 'streaming'
   - n_starts: More for complex models, fewer for simple ones
   - sampler: 'sobol' for better coverage, 'lhs' for efficiency
   - tolerances: Tighter for structural params, looser for rates

5. Override individual parameters:
   fit(model, x, y, workflow='standard', gtol=1e-12)

